In [ ]:
# Copyright 2021 Google LLC.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#3. ML Data Preprocessing

This notebook demonstrates the preparation of an already created ML dataset for model development. It is vital to split machine learning datasets in such a way that the model performance can be tuned and fairly assessed. This notebook shows an example of dividing a dataset into `out-of-time TEST` dataset (including selected full snapshot/s) and `DEVELOPMENT` dataset (randomly splitting the rest of the snapshots into `TRAIN`,`VALIDATION` and `TEST`). Those names are designed to be directly used in the AUTOML [DATA_SPLIT_COL](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-automl#data_split_col).

## Requirements
1. Using [ML Windowing Pipeline (MLWP)](https://github.com/google/gps_building_blocks/tree/master/py/gps_building_blocks/ml/data_prep/ml_windowing_pipeline) to create features tables.

### Imports

In [ ]:
# !pip install gps_building_blocks

In [ ]:
import google.auth
from google.cloud import bigquery
from gps_building_blocks.cloud.utils import bigquery as bigquery_utils

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

### Settings

In [ ]:
# Prints all the outputs from cell (instead of using display each time).
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Configuration

In [ ]:
# Project we are working in.
PROJECT_ID = 'project-id'

# Name of the BigQuery dataset with MLWP tables.
DATASET = "dataset"

# Initial mwlp tables.
TABLE_FEATURES = 'features_01'

# ML datasets
# These 4 tables will be created in {DATASET_DESTINATION}
TABLE_DS_SPLIT = 'ds_split'
TABLE_DS_TRAINING_BALANCED = 'ds_training_balanced'
TABLE_DS_TEST ='ds_test_table'
TABLE_DS_DEV ='ds_dev_table'

In [ ]:
# Initialize BigQuery client.
bq_client = bigquery_utils.BigQueryUtils(project_id=PROJECT_ID)

## Check feature dataset.

1. Determine the right splitting strategy.
2. Verify the date to use as a cut-off for the `OUT-OF-TIME TEST` dataset based on the positive rate trends.
3. Check imbalance in the dataset and decide on a balancing strategy.
4. Consider additional filtering of training data based on snapshot dates.
5. Consider selecting a subset of the columns.


In [ ]:
# Check list of columns to investigate what features are available,
# and potentially selecting a subset of them.
sql = f"""
SELECT
 *
FROM
  `{DATASET}.{TABLE_FEATURES}`
LIMIT 10;
"""
print (sql)
df_raw = bq_client.run_query(sql).to_dataframe()
df_raw.head()

In [ ]:
df_raw.info()

### Check proportion of positive instances in the features table.

In [ ]:
# Produce summary of labels (positive/negative).
sql = f"""
SELECT
  EXTRACT(DATE FROM snapshot_ts) AS effective_date,
  COUNT(*) AS total_records,
  COUNTIF(label=TRUE) AS count_positive,
  COUNTIF(label=FALSE) AS count_negative,
  SAFE_DIVIDE(COUNTIF(label=TRUE),
  COUNTIF(label=FALSE)) AS ratio_positive_to_negative
FROM
  `{DATASET}.{TABLE_FEATURES}`
GROUP BY
  1
ORDER BY
  1 DESC;
"""
print (sql)
df_raw = bq_client.run_query(sql).to_dataframe()
df_raw.head()

In [ ]:
df_features_check = df_raw.copy(deep=True)

In [ ]:
fig = px.line(df_features_check,
              x='effective_date',
              y='ratio_positive_to_negative',
              hover_name='count_positive',
              title='Positive instances',
              height=800)
fig.show()
df_features_check.head(20)

## Create dataset with split on snapshot dates.

#### Get recent effective dates.

In [ ]:
# TODO(): Change split method to use start_date and end_date.
# Get last effective dates.
n_last_dates = 3
recent_dates = df_features_check['effective_date'].sort_values(
    ascending=False).head(n_last_dates).values

In [ ]:
# Keep this if you want to use data driven values for last dates in the dataset.
test_dates = [str(x) for x in recent_dates]

# Define dates here if you want to overwrite with curated dates.
# It is useful to keep looking at a date that was used when
# evaluating original model so we can make sure all performs as expected.
# test_dates = ('2021-05-15', '2021-05-09')
if len(test_dates) == 1:
  test_dates = f"('{tuple(test_dates)[0]}')"
else:
  test_dates = tuple(test_dates)
test_dates

In [ ]:
# Create the dataset if it doesn't exist.
# TODO(): Fix the dataset creation with bq_client(utils version).
dataset = bqclient.create_dataset(PROJECT_ID + '.' + DATASET, exists_ok=True)
print(f'https://console.cloud.google.com/bigquery?project={PROJECT_ID}&p={PROJECT_ID}&d={DATASET_DESTINATION}&page=dataset')

### Create dataset with columns indicating allocation to TRAIN/VALIDATE/TEST.

In [ ]:
sql = f"""
# Add additional columns to dataset to indicate which rows are in which part (train/validate/test).
# Compliant with automl split conventions.
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.{TABLE_DS_SPLIT}` AS (
WITH
  ds_features_key AS (
  SELECT
    *,
    FARM_FINGERPRINT(user_id) AS key,
  FROM
    `{PROJECT_ID}.{DATASET}.{TABLE_FEATURES}`)
SELECT
*,
  CASE
    WHEN EXTRACT(DATE FROM TIMESTAMP (snapshot_ts)) in {test_dates} THEN 'TEST'
    WHEN EXTRACT(DATE FROM TIMESTAMP (snapshot_ts)) not in {test_dates} and MOD(ABS(key), 10) in (0,1,2,3,4,5,6,7) THEN 'TRAIN'
    WHEN EXTRACT(DATE FROM TIMESTAMP (snapshot_ts)) not in {test_dates} and MOD(ABS(key), 10) in (8) THEN 'VALIDATE'
    WHEN EXTRACT(DATE FROM TIMESTAMP (snapshot_ts)) not in {test_dates} and  MOD(ABS(key), 10) in (9) THEN 'TEST'
END as data_split,
FROM
  ds_features_key
  );
"""
print (sql)
df_raw = bq_client.run_query(sql).to_dataframe()
df_raw.head()

## Create TEST (OUT-OF-TIME) dataset

In [ ]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.{TABLE_DS_TEST}` AS (
SELECT * FROM
`{PROJECT_ID}.{DATASET}.{TABLE_DS_SPLIT}`
WHERE EXTRACT(DATE FROM TIMESTAMP (snapshot_ts)) in {test_dates}
);
"""
print (sql)
df_raw = bq_client.run_query(sql).to_dataframe()
df_raw.head()

## Create DEVELOPMENT (IN-TIME) dataset

In [ ]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.{TABLE_DS_DEV}` AS (
SELECT * FROM
`{PROJECT_ID}.{DATASET}.{TABLE_DS_SPLIT}`
WHERE EXTRACT(DATE FROM TIMESTAMP (snapshot_ts)) not in {test_dates}
);
"""
print (sql)
df_raw = bq_client.run_query(sql).to_dataframe()
df_raw.head()

## Optional: Create balanced Train dataset.

This optional step could be used to balance the instances when the % of positive examples in a dataset is very small (for example, less than 1%). When balancing, we will only balance the TRAIN partition of the TABLE_DS_DEV and keep both the VALIDATE and TEST partitions in their original imbalance ratio reflecting the natural data distribution.

Below SQL script keeps all positive instances and then randomly samples records with negative labels; balance_ratio is a parameter we set to control how many times larger is the number of negative instances comparing to the number of positive instances in the resulting dataset. For example, balance_ratio=99 would mean that for every positive instance, we are sampling 99 negative instances.

In [ ]:
# TODO(): Change the way we parametrize the ratio and
# test on different imbalance cases.
balance_ratio = 99
sql = f"""
# Checks imbalance in features by grouping on train/test/validate dataset.
# Automatically balances and downsamples dataset and create one for training.
CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{DATASET}.{TABLE_DS_TRAINING_BALANCED}` AS (
WITH
  dataset AS (
  SELECT
    *
  FROM
    `{PROJECT_ID}.{DATASET}.{TABLE_DS_SPLIT}`
  WHERE
    data_split='TRAIN' ),
  imbalance_stats AS (
  SELECT
    data_split,
    COUNT(*) AS total_records,
    COUNTIF(label=TRUE) AS count_positive,
    COUNTIF(label=FALSE) AS count_negative,
    FORMAT("%.5f", COUNTIF(label=TRUE)/COUNTIF(label=FALSE)) AS ratio_positive_to_negative
  FROM
    dataset
  GROUP BY
    1
  ORDER BY
    1),
  -- Add a random seed to the negative examples.
  negatives AS (
  SELECT
    *,
    RAND() AS rand_seed
  FROM
    dataset
  WHERE
    label = FALSE),
  negative_sampled AS (
  SELECT
    * EXCEPT (rand_seed)
  FROM
    negatives
  WHERE
    -- Here we are sampling the negatives using designed proportion.
    rand_seed <= (
    SELECT
      CAST(ratio_positive_to_negative AS float64)
    FROM
      imbalance_stats)*{balance_ratio} )
  -- Union all together.
SELECT
  *
FROM
  negative_sampled
UNION ALL
  -- Add all positive examples.
SELECT
  *
FROM
  dataset
WHERE
  label = TRUE AND data_split = 'TRAIN'
  );
"""
print (sql)
df_raw = bq_client.run_query(sql).to_dataframe()
df_raw.head()

### Check label balance in the new traning balanced dataset.

In [ ]:
# Produce summary of labels(positive/negative).
sql = f"""
SELECT
  snapshot_ts,
  COUNT(*) AS total_records,
  COUNTIF(label=TRUE) AS count_positive,
  COUNTIF(label=FALSE) AS count_negative,
  FORMAT("%.7f", SAFE_DIVIDE(COUNTIF(label=TRUE),COUNTIF(label=FALSE))) AS ratio_positive_to_negative,
  FORMAT("%.7f", SAFE_DIVIDE(COUNTIF(label=TRUE), COUNT(*))) AS ratio_positive
FROM
  `{DATASET}.{TABLE_DS_TRAINING_BALANCED}`
GROUP BY
  1
ORDER BY
  1 DESC;
"""

print (sql)
df_raw = bq_client.run_query(sql).to_dataframe()
df_raw.head()